Task 1: Basics

In [12]:
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import json
from tqdm import tqdm
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [8]:
# Map your class index to the real ImageNet label index
with open("./TestDataSet/labels_list.json") as f:
    label_list = json.load(f)

class_to_imagenet_idx = {i: int(entry.split(":")[0]) for i, entry in enumerate(label_list)}


In [9]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder("./TestDataSet", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = torchvision.models.resnet34(weights='IMAGENET1K_V1').to(device).eval()

In [10]:
def evaluate_topk(model, dataloader, topk=(1, 5)):
    model.eval()
    top1_correct, top5_correct, total = 0, 0, 0
    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, pred_top5 = outputs.topk(5, 1, True, True)

            # Map ground truth labels to real ImageNet indices
            mapped_labels = torch.tensor(
                [class_to_imagenet_idx[l.item()] for l in labels], device=device
            )

            pred_top1 = pred_top5[:, 0]
            top1_correct += (pred_top1 == mapped_labels).sum().item()
            top5_correct += sum([mapped_labels[i] in pred_top5[i] for i in range(len(mapped_labels))])
            total += labels.size(0)
    return top1_correct / total, top5_correct / total


top1_clean, top5_clean = evaluate_topk(model, test_loader)
print(f"Top-1 Accuracy: {top1_clean:.4f}, Top-5 Accuracy: {top5_clean:.4f}")


100%|██████████| 16/16 [00:35<00:00,  2.21s/it]

Top-1 Accuracy: 0.7600, Top-5 Accuracy: 0.9420


Task 2: Pixel-wise attacks

In [13]:
def fgsm_attack(model, images, labels, epsilon):
    images = images.clone().detach().to(device)
    labels = labels.to(device)
    images.requires_grad = True
    outputs = model(images)
    loss = F.cross_entropy(outputs, labels)
    model.zero_grad()
    loss.backward()
    adv_images = images + epsilon * images.grad.sign()
    return torch.clamp(adv_images, 0, 1).detach()

In [14]:
adv_images_fgsm = []
true_labels = []
for imgs, lbls in tqdm(test_loader):
    adv = fgsm_attack(model, imgs, lbls, epsilon=0.02)
    adv_images_fgsm.append(adv)
    true_labels.append(lbls)

adv_dataset1 = torch.cat(adv_images_fgsm)
true_labels1 = torch.cat(true_labels)

100%|██████████| 16/16 [01:34<00:00,  5.93s/it]


In [15]:
fgsm_loader = DataLoader(torch.utils.data.TensorDataset(adv_dataset1, true_labels1), batch_size=32)
top1_fgsm, top5_fgsm = evaluate_topk(model, fgsm_loader)
print(f"FGSM Top-1: {top1_fgsm:.4f}, Top-5: {top5_fgsm:.4f}")

100%|██████████| 16/16 [00:32<00:00,  2.02s/it]

FGSM Top-1: 0.4320, Top-5: 0.6320


In [16]:
max_diff = torch.max(torch.abs(adv_dataset1 - torch.cat([imgs.to(device) for imgs, _ in test_loader])))
print(f"L∞ max pixel diff: {max_diff:.5f}")

L∞ max pixel diff: 2.11790


In [18]:
def denormalize(img):
    mean = torch.tensor([0.485, 0.456, 0.406], device=img.device)
    std = torch.tensor([0.229, 0.224, 0.225], device=img.device)
    return img * std[:, None, None] + mean[:, None, None]

model.eval()
count = 0
for i in range(len(adv_dataset1)):
    orig = test_dataset[i][0].to(device)
    adv = adv_dataset1[i].to(device)
    label = true_labels1[i].to(device)

    pred_orig = model(orig.unsqueeze(0)).argmax(dim=1)
    pred_adv = model(adv.unsqueeze(0)).argmax(dim=1)

    if pred_orig == label and pred_adv != label:
        fig, axs = plt.subplots(1, 2, figsize=(6, 3))
        axs[0].imshow(denormalize(orig).permute(1, 2, 0).cpu().numpy())
        axs[0].set_title(f"Original: {pred_orig.item()}")
        axs[1].imshow(denormalize(adv).permute(1, 2, 0).cpu().numpy())
        axs[1].set_title(f"Adversarial: {pred_adv.item()}")
        for ax in axs:
            ax.axis('off')
        plt.tight_layout()
        plt.show()
        count += 1
        if count == 5:
            break


Task 3: Improved attacks

In [ ]:
def pgd_attack(model, images, labels, epsilon, alpha, iters):
    ori_images = images.clone().detach().to(device)
    images = ori_images.clone().detach()
    labels = labels.to(device)
    for i in range(iters):
        images.requires_grad = True
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        model.zero_grad()
        loss.backward()
        adv_images = images + alpha * images.grad.sign()
        eta = torch.clamp(adv_images - ori_images, min=-epsilon, max=epsilon)
        images = torch.clamp(ori_images + eta, 0, 1).detach()
    return images

adv_images_pgd = []
for imgs, lbls in tqdm(test_loader):
    adv = pgd_attack(model, imgs, lbls, epsilon=0.02, alpha=0.005, iters=5)
    adv_images_pgd.append(adv)

adv_dataset2 = torch.cat(adv_images_pgd)
pgd_loader = DataLoader(torch.utils.data.TensorDataset(adv_dataset2, true_labels1), batch_size=32)
top1_pgd, top5_pgd = evaluate_topk(model, pgd_loader)
print(f"PGD Top-1: {top1_pgd:.4f}, Top-5: {top5_pgd:.4f}")


Task 4: Patch attacks

In [ ]:
def patch_attack(model, images, labels, epsilon, patch_size=32):
    images = images.clone().detach().to(device)
    labels = labels.to(device)
    B, C, H, W = images.size()
    x = np.random.randint(0, H - patch_size)
    y = np.random.randint(0, W - patch_size)
    patch = images[:, :, x:x+patch_size, y:y+patch_size].clone().detach()
    patch.requires_grad = True
    outputs = model(images)
    loss = F.cross_entropy(outputs, labels)
    model.zero_grad()
    loss.backward()
    adv_patch = patch + epsilon * patch.grad.sign()
    images[:, :, x:x+patch_size, y:y+patch_size] = torch.clamp(adv_patch, 0, 1)
    return images.detach()

adv_images_patch = []
for imgs, lbls in tqdm(test_loader):
    adv = patch_attack(model, imgs, lbls, epsilon=0.3, patch_size=32)
    adv_images_patch.append(adv)

adv_dataset3 = torch.cat(adv_images_patch)
patch_loader = DataLoader(torch.utils.data.TensorDataset(adv_dataset3, true_labels1), batch_size=32)
top1_patch, top5_patch = evaluate_topk(model, patch_loader)
print(f"Patch Attack Top-1: {top1_patch:.4f}, Top-5: {top5_patch:.4f}")


Task 5: Transferring attacks

In [ ]:
densenet = torchvision.models.densenet121(weights='IMAGENET1K_V1').to(device).eval()

top1_clean_dn, top5_clean_dn = evaluate_topk(densenet, test_loader)
top1_fgsm_dn, top5_fgsm_dn = evaluate_topk(densenet, fgsm_loader)
top1_pgd_dn, top5_pgd_dn = evaluate_topk(densenet, pgd_loader)
top1_patch_dn, top5_patch_dn = evaluate_topk(densenet, patch_loader)

print("DenseNet-121 Transferability:")
print(f"Clean      → Top-1: {top1_clean_dn:.4f}, Top-5: {top5_clean_dn:.4f}")
print(f"FGSM       → Top-1: {top1_fgsm_dn:.4f}, Top-5: {top5_fgsm_dn:.4f}")
print(f"PGD        → Top-1: {top1_pgd_dn:.4f}, Top-5: {top5_pgd_dn:.4f}")
print(f"Patch      → Top-1: {top1_patch_dn:.4f}, Top-5: {top5_patch_dn:.4f}")
